<a href="https://colab.research.google.com/github/unclepete-20/lab7-k-means/blob/main/K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio #7 (K-Means)

## Integrantes:


*   Pedro Pablo Arriola Jimenez (20188)
*   Oscar Fernando Lopez Barrios (20679)
*   Santiago Taracena Puga (20017)
*   YongBum Park (20117)








# Introducción a K-Means Clustering para Análisis de Transacciones Bancarias 💳💰

El análisis de transacciones bancarias es una tarea importante para detectar fraudes, comportamientos inusuales y patrones de gasto de los clientes. Una forma de analizar estos datos es mediante el uso de técnicas de agrupamiento, como el algoritmo de K-Means.

El algoritmo de K-Means es una técnica de aprendizaje no supervisado que permite agrupar datos en clusters o grupos, basado en su similitud. En el caso del análisis de transacciones bancarias, se pueden agrupar los datos según el comportamiento de los clientes, como sus patrones de gasto, lugares frecuentes de uso de tarjeta, entre otros.

La implementación de K-Means en Python es relativamente sencilla gracias a la disponibilidad de diversas librerías como scikit-learn y pandas. A través del uso de esta técnica, se puede obtener una mejor comprensión de los patrones y comportamientos de los clientes, lo que puede ser valioso para la toma de decisiones en el ámbito bancario.

En esta investigación, se explorará el uso de K-Means Clustering para el análisis de transacciones bancarias, incluyendo su implementación en Python y la interpretación de los resultados obtenidos.


## Task 1 - Limpieza de datos y Análisis Exploratorio

In [ ]:
# Librerías necesarias para la limpieza de datos y análisis exploratorio.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Se carga el dataset para comenzar a realizar limpieza y exploración.
data = pd.read_csv("./data/bank_transactions.csv")
data.head()

La tabla muestra información sobre transacciones financieras. A continuación se describe el significado de cada columna:

- TransactionID: un identificador único para cada transacción
- CustomerID: un identificador único para cada cliente
- CustomerDOB: la fecha de nacimiento del cliente
- CustGender: el género del cliente
- CustLocation: la ubicación geográfica del cliente
- CustAccountBalance: el saldo de la cuenta del cliente
- TransactionDate: la fecha de la transacción
- TransactionTime: la hora de la transacción
- TransactionAmount (INR): la cantidad de la transacción en rupias indias (INR).

Con esta información, se procederá con la limpieza y codificación de los datos.

In [ ]:
# Se eliminar estas variables categóricas que no sirven ningún propósito.
data = data.drop(["TransactionID", "CustomerID", "CustomerDOB", "TransactionDate"], axis=1)
data.head()

In [ ]:
# Se hace un mapeo para codificar el genero del cliente

gender_map = {"M": 1, "F": 0}  # Mapeo de género a valores numéricos
data["CustGender"] = data["CustGender"].replace(gender_map)
# Convertimos la columna a valores numéricos
data["CustGender"] = pd.to_numeric(data["CustGender"], errors="coerce")
data

In [ ]:
# Se eliminan datos faltantes o nulos
data = data.dropna()
data

In [ ]:
# Escalamiento de la data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[["CustAccountBalance", "TransactionAmount (INR)"]] = scaler.fit_transform(data[["CustAccountBalance", "TransactionAmount (INR)"]])
data

In [ ]:
median_balance = data[["CustAccountBalance"]].median()
fill_median = data[["CustAccountBalance"]].fillna(median_balance)
data.loc[:, "CustAccountBalance"] = fill_median
data

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data.loc[:, "CustLocation"] = le.fit_transform(data["CustLocation"])
data


In [ ]:
# Ahora se hace una breve descripcion estadistica de los datos
data.describe()

In [ ]:
# Tambien se obtiene un poco sobre la informacion de los datos
data.info()

In [ ]:
# Se explora la cantida de ubicaciones para tener una nocion sobre la variable
data.groupby("CustLocation").size()

## Task 1.1 - K-Mean Clustering

In [ ]:
# Comenzamos seleccionando las variables que nos interesan.
X = data[["CustAccountBalance", "TransactionAmount (INR)"]]

# Se grafican los puntos
plt.scatter(X["CustAccountBalance"], X["TransactionAmount (INR)"], c="red")
plt.xlabel("CustAccountBalance")
plt.ylabel("TransactionAmount (INR)")
plt.show()

In [ ]:
# Definir la función de distancia euclidiana
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

# Implementar el método del codo para K-Means Clustering
def kmeans_elbow_method(X, k):
    # Inicializar los centroides de manera aleatoria
    centroids = X.iloc[np.random.choice(X.shape[0], size=k, replace=False), :]
    
    # Iterar sobre diferentes valores de k y calcular SSE
    sse = []
    for i in range(1, k+1):
        clusters = [[] for _ in range(i)]
        for x in X:
            distances = [euclidean_distance(x, c) for c in centroids]
            cluster_idx = np.argmin(distances)
            clusters[cluster_idx].append(x)
        curr_sse = 0
        for j in range(i):
            cluster = np.array(clusters[j])
            centroid = np.mean(cluster, axis=0)
            curr_sse += np.sum((cluster - centroid) ** 2)
        sse.append(curr_sse)
    
    # Graficar SSE vs k y encontrar el codo
    import matplotlib.pyplot as plt
    plt.plot(range(1, k+1), sse, "bx-")
    plt.xlabel("k")
    plt.ylabel("SSE")
    plt.title("Método del codo para K-Means Clustering")
    plt.show()

In [ ]:
# Step 1 and 2 - Choose the number of clusters (k) and select random centroid for each cluster

#number of clusters
K=3

# Select random observation as centroids
Centroids = (X.sample(n=K))
plt.scatter(X["CustAccountBalance"], X["TransactionAmount (INR)"], c="red")
plt.scatter(Centroids["CustAccountBalance"], Centroids["TransactionAmount (INR)"], c="black")
plt.xlabel("CustAccountBalance")
plt.ylabel("TransactionAmount (INR)")
plt.show()

In [ ]:
# Step 3 - Assign all the points to the closest cluster centroid
# Step 4 - Recompute centroids of newly formed clusters
# Step 5 - Repeat step 3 and 4

diff = 1
j=0

while(diff!=0):
    XD=X
    i=1
    for index1,row_c in Centroids.iterrows():
        ED=[]
        for index2,row_d in XD.iterrows():
            d1=(row_c["CustAccountBalance"]-row_d["CustAccountBalance"])**2
            d2=(row_c["TransactionAmount (INR)"]-row_d["TransactionAmount (INR)"])**2
            d=np.sqrt(d1+d2)
            ED.append(d)
        X[i]=ED
        i=i+1

    C=[]
    for index,row in X.iterrows():
        min_dist=row[1]
        pos=1
        for i in range(K):
            if row[i+1] < min_dist:
                min_dist = row[i+1]
                pos=i+1
        C.append(pos)
    X["Cluster"]=C
    Centroids_new = X.groupby(["Cluster"]).mean()[["TransactionAmount (INR)","CustAccountBalance"]]
    if j == 0:
        diff=1
        j=j+1
    else:
        diff = (Centroids_new["TransactionAmount (INR)"] - Centroids["TransactionAmount (INR)"]).sum() + (Centroids_new["CustAccountBalance"] - Centroids["CustAccountBalance"]).sum()
        print(diff.sum())
    Centroids = X.groupby(["Cluster"]).mean()[["TransactionAmount (INR)","CustAccountBalance"]]

In [ ]:
color=["yellow","green","blue"]
for k in range(K):
    data=X[X["Cluster"] == k + 1]
    plt.scatter(data["CustAccountBalance"],data["TransactionAmount (INR)"], c=color[k])
plt.scatter(Centroids["CustAccountBalance"],Centroids["TransactionAmount (INR)"], c="black")
plt.xlabel("Account Balance")
plt.ylabel("TransactionAmount (INR)")
plt.show()

### Task 1.2 - K-Means Clustering con librería

In [ ]:
# Comenzamos importando las librerías importantes para implementar dicho modelo
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(data)
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize = (10, 8))
plt.plot(range(1, 6), pca.explained_variance_ratio_.cumsum(), marker = "o", linestyle = "--")
plt.title("Explained variance ratio by Component")
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance Ratio")
plt.show()

In [ ]:
pca = PCA(n_components=2)
pca.fit(data)
pca_scores = pca.transform(data)

pca_scores

### Método del codo para determinar el número de clusters

In [ ]:
# fitting multiple k-means algorithms and storing the values in an empty list
SSE = []
for cluster in range(1,21):
    kmeans = KMeans(n_clusters = cluster, init="k-means++", random_state= 42)
    kmeans.fit(pca_scores)
    SSE.append(kmeans.inertia_)

plt.figure(figsize=(12,6))
plt.plot(range(1, 21), SSE, marker="o", linestyle="--")
plt.xlabel("Number of clusters")
plt.ylabel("SSE")
plt.title("K-Means Elbow method")
plt.show()

In [ ]:
# k means using 5 clusters and k-means++ initialization
kmeans = KMeans(n_clusters = 3, init="k-means++", random_state=42)
kmeans.fit(data)
pred = kmeans.predict(data)

np.unique(pred)

In [ ]:
data["Segment K-Means"] = kmeans.labels_
data.head()

In [ ]:
data["Segment"] = data["Segment K-Means"].map({0:"First", 1:"Second", 2:"Third"})
data.head()

In [ ]:
x_axis = data["TransactionAmount (INR)"]
y_axis = data["TransactionTime"]
plt.figure(figsize=(12, 8))
sns.scatterplot(x=x_axis, y=y_axis, hue=data["Segment"], palette=["g", "r", "b"])
plt.xlabel("Transaction Amount (INR)")
plt.ylabel("Transaction Time")
plt.title("Clusters by K-Means Clustering")
plt.show()

### Rendimiento de K-Means Clustering de libreria

In [ ]:
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

final_data = data.drop(["Segment", "Segment K-Means"], axis=1)

# calculate metrics
inertia = kmeans.inertia_
db = davies_bouldin_score(final_data, kmeans.labels_)
ch = calinski_harabasz_score(final_data, kmeans.labels_)

print(f"Inertia: {inertia}\nDavies-Bouldin Index: {db}\nCalinski-Harabasz Index: {ch}")